In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("maartengr/arxiv_nlp")["train"]


In [ ]:
abstracts = dataset["Abstracts"]
titles = dataset["Titles"]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('thenlper/gte-small')
embeddings = model.encode(abstracts)

In [ ]:
from umap import UMAP

umap_model = UMAP(n_components=5, min_dist = 0.0, metric ='cosine', random_state= 42)
reduced_embeddigs= umap_model.fit_transform(embeddings)

In [ ]:
from hdbscan import HDBSCAN

hdsbscan_model = HDBSCAN(min_cluster_size = 50, metric = "euclidean", cluster_selection_method= "eom").fit(reduced_embeddigs)
clusters= hdsbscan_model.labels_

In [ ]:
import numpy as np
cluster = 0
for index in np.where(clusters == cluster)[0][:3]:
  print(abstracts[index][:200]+ "...\n")

In [ ]:
from umap import UMAP

reduced_embeddings2 = UMAP( n_components= 2, min_dist =0.0 , metric ="cosine", random_state =42).fit_transform(embeddings)

In [ ]:
import pandas as pd
df = pd.DataFrame(reduced_embeddings2, columns = ["x", "y"])

df["titles"]= titles
df["cluster"]= [str(c) for c in clusters]
to_plot = df.loc[df.cluster != "1", :]
outliers = df.loc[df.cluster == "-1",:]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(outliers.x, outliers.y, alpha= 0.05, s=2, c="grey")
plt.scatter(to_plot.x, to_plot.y, c = to_plot.cluster.astype(int), alpha =0.6, s=2, cmap ="tab20b")

plt.axis("off")